## Stored Procs To Load Data To Bronze Tables:

In [0]:
CREATE OR REPLACE PROCEDURE load_products_from_raw_to_bronze()
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Load products from raw to bronze'
AS
BEGIN
  TRUNCATE TABLE raw_dlk.blinkit_orders.products; 
  INSERT INTO raw_dlk.blinkit_orders.products 
  SELECT 
    * EXCEPT(_rescued_data)
  FROM 
    read_files(
      '/Volumes/raw_dlk/blinkit_orders/sales_data/blinkit_products.csv', 
      format => 'csv'
    )
  WHERE category IN ('Fruits & Vegetables', 'Dairy & Breakfast');
  
END;

In [0]:
CREATE OR REPLACE PROCEDURE load_order_items_from_raw_to_bronze()
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Load order items data from raw to bronze'
AS
BEGIN
  TRUNCATE TABLE raw_dlk.blinkit_orders.order_items; 
  INSERT INTO raw_dlk.blinkit_orders.order_items 
  SELECT 
    * EXCEPT(_rescued_data)
  FROM 
    read_files(
      '/Volumes/raw_dlk/blinkit_orders/sales_data/blinkit_order_items.csv', 
      format => 'csv'
    )
  ;
  
END;

## SQL Program To Generate Recipe For Customers:

In [0]:
CREATE OR REPLACE PROCEDURE load_recipe_to_landing()
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Load recipe data to landing table'
AS
BEGIN
  DROP TABLE IF EXISTS landing.recipe_users.suggested_recipe_users ; 
  CREATE TABLE landing.recipe_users.suggested_recipe_users AS
  SELECT
    *,
    AI_GEN(
      CONCAT(
        'Please suggest and generate a recipe with ingredients: ',
        ORDERED_ITEMS
      )
    ) AS SUGGESTED_RECIPE
  FROM (
    SELECT
      customer_id AS CUSTOMER_ID,
      COUNT(1) AS TOTAL_ORDERS,
      COUNT(DISTINCT product_name) AS TOTAL_UNIQUE_ORDERS,
      STRING_AGG(DISTINCT product_name, ' , ') AS ORDERED_ITEMS
    FROM
      raw_dlk.blinkit_orders.order_items oi
      INNER JOIN raw_dlk.blinkit_orders.products p
        ON oi.product_id = p.product_id
      INNER JOIN raw_dlk.blinkit_orders.orders o
        ON oi.order_id = o.order_id
    GROUP BY 1
    HAVING COUNT(DISTINCT product_name) > 1
    ORDER BY 2 DESC
    LIMIT 10
  ) ; 
END

### Translate Cooking Recipe To Regional Language:

In [0]:

CREATE OR REPLACE PROCEDURE translate_recipe_to_regional_lang(
  IN p_translation STRING  )
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Load recipe data to landing regional language table'
AS
BEGIN
  DROP TABLE IF EXISTS landing.recipe_users.suggested_recipe_users_regional ; 
  CREATE TABLE landing.recipe_users.suggested_recipe_users_regional AS
    SELECT
       CUSTOMER_ID,
     TOTAL_ORDERS,
      TOTAL_UNIQUE_ORDERS,
      ORDERED_ITEMS , 
      ai_translate(
        SUGGESTED_RECIPE,
        p_translation) recipe_in_regional
    FROM
     landing.recipe_users.suggested_recipe_users
 ; 
END


## Bronze Tables:

In [0]:
SELECT * 
FROM 
raw_dlk.blinkit_orders.products 
LIMIT 5 

product_id,product_name,category,brand,price,mrp,margin_percentage,shelf_life_days,min_stock_level,max_stock_level
153019,Onions,Fruits & Vegetables,Aurora LLC,947.95,1263.93,25.0,3,13,88
11422,Potatoes,Fruits & Vegetables,Ramaswamy-Tata,127.16,169.55,25.0,3,20,65
669378,Potatoes,Fruits & Vegetables,Chadha and Sons,212.14,282.85,25.0,3,23,70
848226,Tomatoes,Fruits & Vegetables,Barad and Sons,209.59,279.45,25.0,3,10,51
890623,Onions,Fruits & Vegetables,"Sangha, Nagar and Varty",354.52,472.69,25.0,3,27,55


In [0]:
SELECT *
FROM 
raw_dlk.blinkit_orders.order_items
LIMIT 5

order_id,product_id,quantity,unit_price
1961864118,642612,3,517.03
1549769649,378676,1,881.42
9185164487,741341,2,923.84
9644738826,561860,1,874.78
5427684290,602241,2,976.55


In [0]:
SELECT *
FROM 
raw_dlk.blinkit_orders.orders
LIMIT 5

order_id,customer_id,order_date,promised_delivery_time,actual_delivery_time,delivery_status,order_total,payment_method,delivery_partner_id,store_id
1961864118,30065862,2024-07-17T08:34:01.000Z,2024-07-17T08:52:01.000Z,2024-07-17T08:47:01.000Z,On Time,3197.07,Cash,63230,4771
1549769649,9573071,2024-05-28T13:14:29.000Z,2024-05-28T13:25:29.000Z,2024-05-28T13:27:29.000Z,On Time,976.55,Cash,14983,7534
9185164487,45477575,2024-09-23T13:07:12.000Z,2024-09-23T13:25:12.000Z,2024-09-23T13:29:12.000Z,On Time,839.05,UPI,39859,9886
9644738826,88067569,2023-11-24T16:16:56.000Z,2023-11-24T16:34:56.000Z,2023-11-24T16:33:56.000Z,On Time,440.23,Card,61497,7917
5427684290,83298567,2023-11-20T05:00:39.000Z,2023-11-20T05:17:39.000Z,2023-11-20T05:18:39.000Z,On Time,2526.68,Cash,84315,2741


## PERSONALIZED RECIPE FOR CUSTOMERS:

In [0]:
SELECT * 
FROM 
landing.recipe_users.suggested_recipe_users

CUSTOMER_ID,TOTAL_ORDERS,TOTAL_UNIQUE_ORDERS,ORDERED_ITEMS,SUGGESTED_RECIPE
22210238,4,3,"Carrots , Curd , Tomatoes","What a great combination! Here's a simple and delicious recipe that incorporates Carrots, Curd, and Tomatoes: **Carrot, Tomato, and Curd Raita Salad** Servings: 4-6 people **Ingredients:** * 2 large Carrots, peeled and grated * 1 cup Curd (plain yogurt) * 2 medium Tomatoes, diced * 1/2 teaspoon Cumin powder * 1/2 teaspoon Coriander powder * Salt, to taste * 2 tablespoons Chopped fresh Cilantro (optional) * 1 tablespoon Lemon juice **Instructions:** 1. In a large bowl, whisk the curd until smooth. 2. Add the grated carrots, diced tomatoes, cumin powder, coriander powder, and salt to the bowl. Mix well. 3. Chill the mixture in the refrigerator for at least 30 minutes to allow the flavors to meld. 4. Just before serving, garnish with chopped cilantro (if using) and a squeeze of lemon juice. 5. Serve the Carrot, Tomato, and Curd Raita Salad as a side dish, or use it as a topping for sandwiches, wraps, or salads. **Tips and Variations:** * You can add some chopped cucumber or bell peppers to the salad for extra flavor and texture. * For a spicy kick, add a pinch of red chili powder or a diced green chili to the mixture. * If you prefer a thicker consistency, you can add some chopped cucumber or a tablespoon of gram flour (besan) to the curd. * This salad is perfect for a light and refreshing snack or as a side dish for Indian-inspired meals. Enjoy your delicious and healthy Carrot, Tomato, and Curd Raita Salad!"
45477575,3,2,"Tomatoes , Eggs","What a great combination! Here's a simple and delicious recipe that combines Tomatoes and Eggs: **Shakshuka with Fresh Tomatoes and Poached Eggs** Servings: 2-3 **Ingredients:** * 3 large Tomatoes, diced * 2 Eggs * 1 small Onion, diced * 2 cloves of Garlic, minced * 1 teaspoon Smoked Paprika (optional) * Salt and Pepper, to taste * 2 tablespoons Olive Oil * Fresh Parsley or Cilantro, chopped (for garnish) **Instructions:** 1. **Heat the oil**: In a large cast-iron skillet or oven-safe pan, heat the olive oil over medium heat. 2. **Sauté the onion and garlic**: Add the diced onion and cook until translucent, about 3-4 minutes. Add the minced garlic and cook for another minute, until fragrant. 3. **Add the tomatoes**: Add the diced tomatoes, smoked paprika (if using), salt, and pepper. Stir well to combine. 4. **Simmer the tomatoes**: Reduce heat to low and let the tomatoes simmer for 10-15 minutes, stirring occasionally, until they start to break down and form a rich, saucy mixture. 5. **Create wells for the eggs**: Use a spoon to create 2-3 wells in the tomato mixture, depending on the number of eggs you're using. 6. **Crack in the eggs**: Crack an egg into each well, making sure not to break the yolk. 7. **Poach the eggs**: Cover the skillet with a lid and let the eggs poach in the tomato mixture for 5-7 minutes, or until the whites are set and the yolks are cooked to your desired doneness. 8. **Garnish and serve**: Garnish with chopped parsley or cilantro, and serve hot with crusty bread or over rice. **Tips and Variations:** * Use fresh, flavorful tomatoes for the best results. * Add some heat with red pepper flakes or diced jalapeños. * Mix in some chopped bell peppers or mushrooms for added flavor and texture. * Serve with a side of toasted bread or pita for a satisfying breakfast or brunch. Enjoy your delicious Shakshuka with Fresh Tomatoes and Poached Eggs!"
10562528,3,3,"Curd , Cheese , Potatoes","What a great combination! Here's a recipe that combines Curd, Cheese, and Potatoes: **Curd and Cheese Stuffed Baked Potatoes** Servings: 4-6 people **Ingredients:** * 4-6 large potatoes * 1 cup curd (plain yogurt) * 1/2 cup grated cheese (Cheddar or Mozzarella work well) * 1/4 cup chopped fresh cilantro (optional) * 1/2 teaspoon salt * 1/4 teaspoon black pepper * 1 tablespoon olive oil **Instructions:** 1. **Preheat the oven:** Preheat the oven to 400°F (200°C). 2. **Prepare 

In [0]:
SELECT * 
FROM 
landing.recipe_users.suggested_recipe_users_regional

CUSTOMER_ID,TOTAL_ORDERS,TOTAL_UNIQUE_ORDERS,ORDERED_ITEMS,recipe_in_regional
22210238,4,3,"Carrots , Curd , Tomatoes","वाह क्या अद्भुत संयोजन है! यहाँ एक सरल और स्वादिष्ट रेसिपी है जिसमें गाजर, दही और टमाटर शामिल हैं: **गाजर, टमाटर और दही रायता सलाद** परोसने की संख्या: ४-६ लोग **सामग्री:** * २ बड़े गाजर, छिलके उतारे और कद्दूकस किए हुए * १ कप दही (सादा दही) * २ मध्यम टमाटर, कटे हुए * १/२ चम्मच जीरा पाउडर * १/२ चम्मच धनिया पाउडर * नमक, स्वादानुसार * २ बड़े चम्मच कटा हुआ ताज़ा धनिया (वैकल्पिक) * १ बड़ा चम्मच नींबू का रस **निर्देश:** १. एक बड़े बाउल में, दही को चिकना होने तक फेंटें। २. बाउल में कद्दूकस किए हुए गाजर, कटे हुए टमाटर, जीरा पाउडर, धनिया पाउडर और नमक मिलाएं। अच्छी तरह मिलाएं। ३. मिश्रण को कम से कम ३० मिनट के लिए फ्रिज में रखें ताकि स्वाद मिल सके। ४. परोसने से पहले, कटा हुआ धनिया (यदि उपयोग कर रहे हैं) और नींबू के रस के साथ गार्निश करें। ५. गाजर, टमाटर और दही रायता सलाद को एक साइड डिश के रूप में परोसें या सैंडविच, रैप या सलाद के ऊपर उपयोग करें। **सुझाव और विविधताएं:** * आप सलाद में कुछ कटा हुआ खीरा या शिमला मिर्च मिला सकते हैं जिससे स्वाद और बनावट बढ़ जाए। * एक तीखा स्वाद के लिए, मिश्रण में एक चुटकी लाल मिर्च पाउडर या एक कटा हुआ हरी मिर्च मिला सकते हैं। * यदि आप एक गाढ़ा स्थिरता पसंद करते हैं, तो आप दही में कुछ कटा हुआ खीरा या एक बड़ा चम्मच बेसन मिला सकते हैं। * यह सलाद एक हल्का और ताज़ा नाश्ता या भारतीय-प्रेरित भोजन के लिए एक साइड डिश के रूप में उपयुक्त है। अपने स्वादिष्ट और स्वास्थ्यवर्धक गाजर, टमाटर और दही रायता सलाद का आनंद लें!"
45477575,3,2,"Tomatoes , Eggs","वाह क्या अद्भुत संयोजन है! यहाँ एक सरल और स्वादिष्ट नुस्खा है जो टमाटर और अंडे को मिलाता है: **ताजे टमाटर और पोच्ड एग्स के साथ शाकशूका** परोसने की संख्या: २-३ **सामग्री:** * ३ बड़े टमाटर, कटे हुए * २ अंडे * १ छोटा प्याज, कटा हुआ * २ लहसुन की कलियाँ, बारीक कटी हुई * १ चम्मच धूम्रपान पाप्रिका (वैकल्पिक) * नमक और काली मिर्च, स्वादानुसार * २ बड़े चम्मच जैतून का तेल * ताजा अजमोद या धनिया, कटा हुआ (गार्निश के लिए) **निर्देश:** १. **तेल गरम करें**: एक बड़े लोहे के तवे या ओवन-सुरक्षित पैन में, मध्यम आंच पर जैतून का तेल गरम करें। २. **प्याज और लहसुन को भूनें**: कटे हुए प्याज को डालें और ३-४ मिनट तक पकाएं, जब तक कि वे पारदर्शी न हो जाएं। बारीक कटी हुई लहसुन को डालें और एक मिनट तक पकाएं, जब तक कि वे सुगंधित न हो जाएं। ३. **टमाटर डालें**: कटे हुए टमाटर, धूम्रपान पाप्रिका (यदि उपयोग कर रहे हैं), नमक, और काली मिर्च डालें। अच्छी तरह मिलाएं। ४. **टमाटर को पकाएं**: आंच को कम करें और टमाटर को १०-१५ मिनट तक पकने दें, कभी-कभी हिलाते हुए, जब तक कि वे टूटने न लगें और एक समृद्ध, सॉसी मिश्रण न बन जाए। ५. **अंडे के लिए गड्ढे बनाएं**: एक चम्मच का उपयोग करके टमाटर के मिश्रण में २-३ गड्ढे बनाएं, अंडे की संख्या के अनुसार। ६. **अंडे तोड़ें**: प्रत्येक गड्ढे में एक अंडा तोड़ें, सुनिश्चित करें कि जर्दी न टूटे। ७. **अंडे को पोच करें**: तवे को ढक्कन से ढकें और अंडे को टमाटर के मिश्रण में ५-७ मिनट तक पकने दें, या जब तक कि सफेद भाग जम जाए और जर्दी आपकी वांछित पकाने की डिग्री तक पक जाए। ८. **गार्निश और परोसें**: कटे हुए अजमोद या धनिया से गार्निश करें, और गरमा गरम परोसें क्रस्टी ब्रेड या चावल के साथ। **सुझाव और विविधताएं:** * सबसे अच्छे परिणामों के लिए ताजे, स्वादिष्ट टमाटर का उपयोग करें। * लाल मिर्च के फ्लेक्स या कटे हुए जलापेनोस के साथ कुछ गर्मी जोड़ें। * कुछ कटे हुए शिमला मिर्च या मशरूम मिलाएं जोड़ने के लिए स्वाद और बनावट। * एक संतोषजनक नाश्ते या दोपहर के भोजन के लिए टोस्टेड ब्रेड या पिटा के साथ परोसें। ताजे टमाटर और पोच्ड एग्स के साथ अपने स्वादिष्ट शाकशूका का आनंद लें!"
10562528,3,3,"Curd , Cheese , Potatoes","वाह क्या अद्भुत संयोजन है! यहाँ एक रेसिपी है जिसमें दही, पनीर और आलू शामिल हैं: **दही और पनीर से भरे हुए बेक्ड आलू** परोसने की संख्या: ४-६ लोग **सामग्री:** * ४-६ बड़े आलू * १ कप दही (सादा दही) * १/२ कप कसा हुआ पनीर (चेडर या मोज्जेरेला अच्छा काम करता है) * १/४ कप कटा हुआ ताज़ा धनिया (वैकल्पिक) * १/२ चम्मच नमक * १/४ चम्मच काली मिर्च * १ बड़ा चम्मच जैतून का तेल **निर्देश:** १. **ओवन को पहले से गरम करें:** ओवन को ४००°F (२००°C) पर गरम करें। २. **आलू तैयार करें:** आलू को साफ करें और एक पेपर तौलिये से सुखाएं। प्रत्येक आलू में एक फोर्क से कुछ छेद करें ताकि भाप निकल सके। ३. **आलू बेक करें:** आलू